In [ ]:
  import math
  import matplotlib.pyplot as plt
  import numpy as np

In [ ]:
def sinc(x):
    if x == 0:
        return 1
    else:
        return math.sin(x)/x

In [ ]:
l=-10
r=10
step=0.1


input=[]
output=[]
while l<=r:
  input.append(l)
  output.append(sinc(l))
  l+=step 

input=np.array(input)
output=np.array(output)

plt.plot(input,output)

In [ ]:
def tansig(x):
    return 2/(1+math.exp(-2*x))-1


In [ ]:
w=np.random.uniform(low=-10,high=10,size=(201,))
x=np.random.uniform(low=-10,high=10,size=(201,))
b=np.random.uniform(low=-10,high=10,size=(201,))
sum=w*x+b

out=np.array(list(map(tansig,sum)))
h=[np.transpose(out)]
h=np.array(h)
h_inv=np.linalg.pinv(h)
b_new=h_inv*output
print(h_inv.shape)
print(output.shape)
# b_new=b_new[0]
# print(b_new.shape)